#### Kaggle: https://www.kaggle.com/c/sai-vessel-segmentation2

In [ ]:
import os
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
from tqdm.auto import tqdm
import imgaug
from sklearn.model_selection import train_test_split
import pandas as pd

import torch
from torch import nn
import torchvision
import torchsummary

torch.__version__, imgaug.__version__

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

#### Download data

In [ ]:
# download dataset from https://drive.google.com/file/d/1JILW10sr40CRTLiuA1mf__5GBtMDF9xc/view?usp=sharing
!pip install --upgrade gdown
!gdown --id 1JILW10sr40CRTLiuA1mf__5GBtMDF9xc --output vessel_seg.zip

In [ ]:
# unzip file
!unzip -q vessel_seg.zip

#### Data Analysis


---

all


*   train/
    *   id1_training.tif
    *   id1**_manual1**.gif
    *   id2_training.tif
    *   id2**_manual1**.gif
    *   ...
*   test/
    *   ...



In [ ]:
# read img and mask
img_paths = sorted(glob('all/train/*.tif'))
img_path = img_paths[0]
mask_path = img_path.replace('_training.tif', '_manual1.gif')

print('img path: ', img_path)
print('mask path:',  mask_path)

img = cv2.imread(img_path)[:, :, ::-1]
mask = Image.open(mask_path)
mask = np.array(mask)

Image shape: (H, W, C)

*   C: 3 RGB channels

Mask  shape: (H, W)
* 0: background
* 255: vessel

In [ ]:
# show image
plt.figure(figsize=(20, 5))
plt.subplot(1,2,1)
plt.imshow(img)
plt.subplot(1,2,2)
plt.imshow(mask)
plt.show()

In [ ]:
img.shape, mask.shape

In [ ]:
# Calculate tumor pixel
values, counts = np.unique(mask, return_counts=True)
values, counts

# Dataset & Dataloader


In [ ]:
IMG_SIZE = 512
BS = 4

In [ ]:
class VesselDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, img_size, mode, augmentation=False):
        self.img_paths = img_paths
        self.mask_paths = [m_path.replace('training.tif', 'manual1.gif') for m_path in self.img_paths]
        self.img_size = img_size
        self.mode = mode
        # Augmentation
        self.augmentation = augmentation
        self.augmentor = imgaug.augmenters.Sequential([
            imgaug.augmenters.Fliplr(0.5), # 50% horizontal flip
            imgaug.augmenters.Affine(
                rotate=(-45, 45), # random rotate -45 ~ +45 degree
                shear=(-16,16), # random shear -16 ~ +16 degree
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)} # scale x, y: 80%~120%
            ),
        ])
    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = cv2.imread(img_path)[:, :, ::-1]
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) # (IMG_SIZE, IMG_SIZE, 3)
        # Normalize Image
        img = img / 255. # 0~255 -> 0~1

        if self.mode != 'test':
            mask_path = self.mask_paths[idx]
            mask = np.array(Image.open(mask_path))
            mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE)) # (IMG_SIZE, IMG_SIZE)
            # Binarize mask from [0~255] to (0 or 1)
            mask = np.where(mask<1, 0, 1).astype(np.int16)

            # Augment mask by imgaug
            if self.augmentation:
                # to imgaug data class
                mask = imgaug.augmentables.segmaps.SegmentationMapsOnImage(mask,
                                                                    shape=mask.shape)
                # augment img & mask "simultaneously"
                img, mask = self.augmentor(image=img, segmentation_maps=mask)
                mask = mask.get_arr() # to np.ndarray

            # 1. Convert to PyTorch Tensor
            # 2. Channel last to first: (H, W, C) -> (C, H, W)
            # Tensor datatype:
            # Img: float, Mask: long
            img = torch.tensor(img, dtype=torch.float).permute(2, 0, 1)
            mask = torch.tensor(mask, dtype=torch.long)
            return img, mask
        else:
            img_tensor = torch.tensor(img, dtype=torch.float).permute(2, 0, 1)
            return img_tensor

In [ ]:
all_paths = sorted(glob('all/train/*.tif'))
test_paths = sorted(glob('all/test/*.tif'))
train_paths, val_paths = train_test_split(all_paths, test_size=0.2)

In [ ]:
train_ds = VesselDataset(train_paths, IMG_SIZE, 'train', augmentation=True)
val_ds = VesselDataset(val_paths, IMG_SIZE, 'val')
test_ds = VesselDataset(test_paths, IMG_SIZE, 'test')

# https://pytorch.org/docs/stable/data.html
NUM_WORKERS = 2 # > 0 to accelerate loading data by muli-process

train_loader = torch.utils.data.DataLoader(train_ds, BS, shuffle=True, num_workers=NUM_WORKERS)
val_loader = torch.utils.data.DataLoader(val_ds, BS, num_workers=NUM_WORKERS)
test_loader = torch.utils.data.DataLoader(test_ds, BS)

Visulize data

In [ ]:
img, mask = train_ds[0] # take 1 data
print('img.shape: ', img.shape, '\nmask.shape: ', mask.shape)
img = img.numpy().transpose(1, 2, 0) # (C, H, W) -> (H, W, C)
mask = mask.numpy().squeeze() # (1, H, W) -> (H, W)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.subplot(1, 2, 2)
plt.imshow(mask)
plt.show()

#### Build Model

In [ ]:
# Basic Conv block
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
    def __call__(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()
        # Encoder
        self.encoder1 = ConvBlock(in_channels, init_features) # (3, H, W) -> (32, H, W)
        self.pool1 = nn.MaxPool2d(2)
        self.encoder2 = ConvBlock(init_features, init_features*2) # (32, H/2, W/2) -> (64, H/2, W/2)
        self.pool2 = nn.MaxPool2d(2)
        self.encoder3 = ConvBlock(init_features*2, init_features*4) # (64, H/4, W/4) -> (128, H/4, W/4)
        self.pool3 = nn.MaxPool2d(2)
        self.encoder4 = ConvBlock(init_features*4, init_features*8) # (128, H/8, W/8) -> (256, H/8, W/8)
        self.pool4 = nn.MaxPool2d(2)
        self.encoder5 = ConvBlock(init_features*8, init_features*16) # (256, H/16, W/16) -> (512, H/16, W/16)

        # Decoder
        self.upconv4 = nn.ConvTranspose2d(init_features*16, # (512, H/16, W/16) -> (256, H/8, W/8)
                          init_features*8,
                          kernel_size=2, stride=2)
        self.decoder4 = ConvBlock(init_features*8*2, init_features*8)
        self.upconv3 = nn.ConvTranspose2d(init_features*8, # (256, H/8, W/8) -> (128, H/4, W/4)
                          init_features*4,
                          kernel_size=2, stride=2)
        self.decoder3 = ConvBlock(init_features*4*2, init_features*4)
        self.upconv2 = nn.ConvTranspose2d(init_features*4, # (128, H/4, W/4) -> (64, H/2, W/2)
                          init_features*2,
                          kernel_size=2, stride=2)
        self.decoder2 = ConvBlock(init_features*2*2, init_features*2)
        self.upconv1 = nn.ConvTranspose2d(init_features*2, # (62, H/2, W/2) -> (32, H, W)
                          init_features,
                          kernel_size=2, stride=2)
        self.decoder1 = ConvBlock(init_features*2, init_features)
        # Output (head): classification for each pixel, C=2
        self.output = nn.Conv2d(
            init_features,
            out_channels=out_channels,
            kernel_size=1) # (32, H, W) -> (2, H, W)

    def forward(self, x):
        # Encoder
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))
        bottleneck = self.encoder5(self.pool4(enc4))

        # Decoder
        # (BS, 256, H/8, W/8) + (BS, 256, H/8, W/8) -> (BS, 512, H/8, W/8)
        x = torch.cat((self.upconv4(bottleneck), enc4), dim=1)
        x = self.decoder4(x)

        x = torch.cat((self.upconv3(x), enc3), dim=1)
        x = self.decoder3(x)

        x = torch.cat((self.upconv2(x), enc2), dim=1)
        x = self.decoder2(x)

        x = torch.cat((self.upconv1(x), enc1), dim=1)
        x = self.decoder1(x)

        x = self.output(x)
        return x

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# build model to GPU
model = UNet(in_channels=3,
             out_channels=2,
             init_features=32).to(device)

#### Training

TODO

Kaggle submission

In [ ]:
model.load_state_dict(torch.load("best.pth"))

In [ ]:
# make prediction
outputs = []
with torch.no_grad():
    for imgs in test_loader:
        imgs = imgs.to(device)
        y_preds = model(imgs).cpu().numpy()
        y_preds = np.argmax(y_preds, axis=1)
        y_preds = np.expand_dims(y_preds, axis=-1)

        for y_pred in y_preds:
            dots = np.where(y_pred.flatten() == 1)[0]
            run_lengths = []
            prev = -2
            for b in dots:
                if (b > prev +1):
                    run_lengths.extend((b+1,0))
                run_lengths[-1] += 1
                prev = b
            output = ' '.join([str(r) for r in run_lengths])
            outputs.append(output)


In [ ]:
# Create submission file
df = pd.DataFrame(columns=['Id', 'Predicted'])
df['Id'] = [str(i) for i in range(20)]
df['Predicted'] = outputs
df.to_csv('submission.csv', index=None)
df